In [110]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import random
from colorama import Fore
from colorama import Style
import time
from datetime import timedelta

edgecount=383842 #number of ratings
usercount=278854 #number of users
bookcount=271361 #number of books

G=nx.Graph()
G_test=nx.Graph()
N=pd.read_csv('final.csv') #input weighted edgelist - column 1 - source, column 2 - target, column 3 - rating
Books=pd.read_csv('new_books.csv') #input books database

for i in range(edgecount): #forming weighted graph
    G.add_node(N.Book[i])
    G.add_node(N.User[i])
    G.add_edge(N.User[i],N.Book[i], weight=N.Rating[i])

In [369]:
print("Enter number of sample users")
n_users=int(input())

Enter number of sample users
5


In [370]:
sample_users1=[]
for x in list(set(N.User)):
    if G.has_node(x):
        if G.degree(x)>=8:
            sample_users1.append(x)
sample_users=random.sample(sample_users1, n_users)

In [374]:
#reco_user=32773
sample_reading_lists = []

for x in sample_users:
    if x in G.nodes():
        avg_rating=G.degree(x, weight='weight')/G.degree(x)
        user_fav=[]
        for i in set(N.Book):
            if G.has_edge(x, i):
                if G[x][i]['weight']>=avg_rating:
                    user_fav.append(i)

        user_fav.sort(reverse=True)
        sample_reading_lists.append(user_fav[:10])

train_list=[]

for i in range(w):
    train_list.append(sample_reading_lists[i][:10])
w=len(sample_reading_lists)
temp_user=list(set(N.User))
temp_user.sort()

#the next cell is where the evaluation happens. It might take a few minutes.

In [375]:
#subgraph formation using G
start_time=time.monotonic()

tp=0
tn=0
fp=0
fn=0
count=0
catalog=0


for y in range(w):
    wt=[]
    row=[0]*(usercount+bookcount+1)
    for i in range(usercount+bookcount+1):
        wt.append(row)
    
    Z=nx.Graph()
    
    for x in train_list[y]:
        for i in temp_user:
            for j in list(G.neighbors(i)): 
                if G.has_edge(i,x) and G.has_edge(i,j) and j!=x:
                    wt[j][x]=wt[j][x]+1
                    Z.add_node(j, node_weight=G.degree(j,weight='weight')/G.degree(j))
                    Z.add_node(x, node_weight=G.degree(x,weight='weight')/G.degree(x))
                    Z.add_edge(j,x, weight=wt[j][x])
#common neighbors    
    temp=train_list[y]
    for i in temp:
        if i not in Z.nodes():
            temp.remove(i)

    to_read1=list(set(Z.neighbors(temp[0])))

    for i in temp:
        to_read1=list(set(Z.neighbors(i)).intersection(to_read1))
    
    tempx=[]
    for x in train_list[y]:
        for q in train_list[y]:
            if G.has_node(x) and G.has_node(q):
                if nx.has_path(G,x,q):
                    temp=list(nx.shortest_path(G, source=x, target=q))
                    tempx.append(temp)
    if len(tempx)>=2:
        to_read2=list(set(tempx[0]).intersection(tempx[1]))

#finding common elements in the shortest paths

        for i in range(len(tempx)):
            to_read2=list(set(tempx[i]).intersection(to_read2))

    to_read=list(set(to_read1) | set(to_read2))
    
    if not to_read:
        to_read=random.sample(list(Z), min(nx.number_of_nodes(Z),2))

    for i in reading_list:
        if i in to_read:
            to_read.remove(i)
            
    for i in to_read:
        if i in list(G.neighbors(sample_users[count])):
            tp=tp+1
        else:
            fp=fp+1
    catalog=catalog+max(10,len(to_read))
    count=count+1  

end_time=time.monotonic()

In [376]:
print("true positive=", tp)
print("false positive=", fp)
print("precision=", 100*tp/(tp+fp))
print(count)
print(timedelta(seconds=end_time-start_time))

true positive= 22
false positive= 3
precision= 88.0
5
0:00:30.203000
